In [ ]:
# how to kill all processes
# sudo kill -9 $(pgrep -f python3)
# sudo kill -9 $(pgrep -f conformanceTests)

# import subprocess
# subprocess.run(["kill" "-9" $(pgrep -f conformanceTests)"])
import os
os.system("kill -9 $(pgrep -f conformanceTests)")

In [ ]:
!pip install xlsxwriter==3.0.2
!pip install beautifulsoup4==4.10.0
!pip install utils==1.0.1

In [ ]:
import os
python_exec = '/home/nsemaev/code/venv/bin/python3.8'

work_path = os.getcwd()

binary_type = 'master_2022_02_15'
binary_name = f"conformanceTests_{binary_type}"
binary_path = f"{work_path}/bin/{binary_name}"


ci_report_file = f"{work_path}/ci_reports/" + "2022_02_15.html"
report_plugin_position = 4

results_path = f"{work_path}/{binary_type}"

ping_time = 60 * 5

cores = 16

irs_path = '/home/nsemaev/Documents/ops/'

skipped_ops = ['boolean', 'ROIAlign']
time_wasting_ops = ['BinaryConvolution', 'Convolution', 'ConvolutionBackpropData', 'DeformableConvolution', 
                    'GroupConvolution', 'GroupConvolutionBackpropData', 'MaxPool', 'AvgPool', 'Interpolate',
                    'TensorIterator', 'MVN', 'Reshape']

all_ops = sorted(os.listdir(irs_path))

run_results = ['passed', 'failed', 'skipped', 'crashed', 'empty']
ci_results = ['passed', 'failed', 'skipped', 'crashed']

In [ ]:
def get_precision_irs_list(operation):
    precision_irs_dict = [{} for i in range(10)]
    for precision in os.listdir(f"{irs_path}/{operation}"):
        assert os.path.isdir(f"{irs_path}/{operation}/{precision}")
        for file_name in os.listdir(f"{irs_path}/{operation}/{precision}"):
            _, file_extension = os.path.splitext(file_name)
            if file_extension == '.xml':
                ir_id = int(re.findall(fr"{operation}_(\d+)\.xml", file_name)[0])
                with open(f"{irs_path}/{operation}/{precision}/{file_name}") as xml_file:
                    xml_data = str(xml_file.read())
                opset = int(re.findall(fr'type="{operation.lower()}" version="opset(\d+)">', xml_data.lower())[0])
                if precision not in precision_irs_dict[opset]:
                    precision_irs_dict[opset][precision] = []
#                 print(opset, type(opset))
                precision_irs_dict[opset][precision].append(ir_id)
    return precision_irs_dict

In [ ]:
get_precision_irs_list('Add')

In [ ]:
import os
from os import path
import sys
import time
from datetime import datetime
import shutil
import subprocess
import _thread
import threading
from contextlib import contextmanager
import re
import xlsxwriter
from bs4 import BeautifulSoup


completed_ops = []
if os.path.exists(results_path):
    completed_ops = [op.split('_')[0] for op in os.listdir(results_path) if op.split('_')[-1] == 'completed']
logs_files = {}

class GTestParallel():
    def __init__(self, operation: str):
        print(f"\n\n\n{operation}")
        self.operation = operation
        self.op_results_path = f"{results_path}/{self.operation}_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
        self.precision_irs_list = get_precision_irs_list(self.operation)
#         self.reports_path = f"{self.op_results_path}_reports"
#         os.makedirs(self.reports_path)
        for opset in range(len(self.precision_irs_list)):
            for precision in self.precision_irs_list[opset]:
                for ir_id in self.precision_irs_list[opset][precision]:
                    dst_folder = f"{self.operation}_opset{opset}_{precision}_{ir_id}"
                    dst_path = f"{self.op_results_path}/{dst_folder}/ir"
                    os.makedirs(dst_path)
                    for extension in ['xml', 'bin', 'meta']:
                        ir_file_name = f"{self.operation}_{ir_id}.{extension}"
                        src_path = f"{irs_path}/{self.operation}/{precision}"
                        shutil.copyfile(f"{src_path}/{ir_file_name}", f"{dst_path}/{ir_file_name}")
               
    def get_command(self, folder: str):
        return f"{python_exec} {work_path}/gtest_parallel.py {binary_path} -d . " + \
            f"--gtest_filter=*ReadIRTest*{self.operation}* " + \
            f"-- " + \
            f"--input_folders={self.op_results_path}/{folder}/ir " + \
            f"--device=TEMPLATE " + \
            f"--output_folder={self.op_results_path}/{folder}"
#             f"--report_unique_name " + \
#             f"--output_folder={self.op_results_path}/{folder}/gtest-parallel-xmls"
    
    def run (self, time_limit):
        completed = 0
        start_time = time.time()
        waiting_time = 10
        for folder in os.listdir(self.op_results_path):
            command = self.get_command(folder)
            threading.Thread(target=os.system, args=(f"cd {self.op_results_path}/{folder} && {command}",)).start()
        while completed < len(os.listdir(self.op_results_path)):
            for folder in os.listdir(self.op_results_path):
                if not folder.endswith('completed'):
                    if os.path.exists(f"{self.op_results_path}/{folder}/gtest-parallel-logs/passed") or \
                       os.path.exists(f"{self.op_results_path}/{folder}/gtest-parallel-logs/failed"):
#                         report_name = f"{folder}_report.xml"
#                         if os.path.exists(f"{self.op_results_path}/{folder}/report.xml"):
#                             os.rename(f"{self.op_results_path}/{folder}/report.xml", 
#                                       f"{self.op_results_path}/{folder}/{report_name}")
#                             shutil.copyfile(f"{self.op_results_path}/{folder}/{report_name}", 
#                                             f"{self.reports_path}/{report_name}")
                        os.rename(f"{self.op_results_path}/{folder}", 
                                  f"{self.op_results_path}/{folder}_{int(time.time() - start_time)}s_completed")
                        completed += 1
            if datetime.now().strftime('%Y_%m_%d %H:%M:%S')[-1] == '0':
                print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.operation} ({time.time() - start_time} of {time_limit})")
                print(f"{completed} of {len(os.listdir(self.op_results_path))}")
            if time.time() - start_time > time_limit:
                os.system('kill -9 $(pgrep -f conformanceTests)')
                break
            time.sleep(1)
        os.rename(f"{self.op_results_path}", 
                  f"{self.op_results_path}_{completed}_of_{len(os.listdir(self.op_results_path))}_{int(time.time() - start_time)}s_completed")
    
    def get_precision_irs_dict(self):
        for precision in os.listdir(f"{irs_path}/{self.op}/"):
            assert os.path.isdir(f"{irs_path}/{self.op}/{precision}/")
            for file_name in os.listsir(f"{irs_path}/{self.op}/{precision}/"):
                _, file_extension = os.path.splitext(file_name)
        
    def generate_tmp_irs_path(self, irs_path: str, ir_id: int):
        os.makedirs(f"{work_path}/tmp/{self.op}_{ir_id}")
        shutil.copyfile(f"{irs_path}/", dst)
        
    

#     def run_while_not_end(self, op_time=30, time_limited=True):
#         process_start_time = time.time()
#         current_time = datetime.now().strftime('%Y_%m_%d %H:%M:%S')
#         print(f"{current_time} {self.op} was started")
#         print(self.command)
#         if os.path.exists(self.op_path):
#             shutil.rmtree(self.op_path)
#         os.makedirs(self.op_path)
#         os.chdir(self.op_path)
#         threading.Thread(target=os.system, args=(f"cd {self.op_path} && {self.command} > /dev/null",)).start()
#         # _thread.start_new_thread(os.system, (f"cd {self.op_path} && nohup {self.command}",))
#         os.chdir(work_path)
#         # statuses = ['passed', 'failed']
#         start_time = time.time()
#         last_ping = time.time()
#         while True:
#             time.sleep(1)
#             if (os.path.exists(f"{self.op_path}/gtest-parallel-logs/passed")):
#                 break
#             if (os.path.exists(f"{self.op_path}/gtest-parallel-logs/failed")):
#                 break
#             if time_limited and (time.time() - start_time > op_time):
#                 print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} Sorry, time for operation {self.op} is over")
#                 os.rename(self.op_path, self.op_stopped_path)
#                 os.system('kill -9 $(pgrep -f conformanceTests)')
#                 return False
#             if time.time() - last_ping > ping_time:
#                 print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.op} ping")
#                 last_ping = time.time()
        
#         log_files_folder = f"{self.op_path}/gtest-parallel-logs/failed"
#         failed_logs_result = f"{self.op_path}/{self.op}_failed_logs_result.txt"
#         if os.path.exists(log_files_folder):
#             with open(failed_logs_result, 'w', encoding='utf-8') as result_file:
#                 for log_file_name in os.listdir(log_files_folder):
#                     log_file_path = os.path.join(log_files_folder, log_file_name)  # !!! везде так же сделать

#                     # print(f"\n\n{log_file_path}\n\n")

#                     with open(log_file_path, 'r', encoding='utf-8') as file:
#                         data = str(file.read())
#                         test_filter = '/'.join(re.findall(r"Google Test filter = ([^\n]+)", data))
#                         mem_usage = '/'.join(re.findall(r"MEM_USAGE=([^\n]+)", data))
#                         failures = '/'.join(re.findall(r"[^\n]+pp:\s*\d+[^\n]+", data))
#                         # print(f"\n\n{test_filter}")
#                         # print(f"\n\n{mem_usage}")
#                         # print(f"\n\n{failures}")
#                         result_file.write(test_filter + ',')
#                         result_file.write(mem_usage + ',')
#                         result_file.write(failures + ';\n')
#                     # with open(log_path, 'r', encoding='utf-8') as file:
#                     #     print(file.read())

#         print(f"cd {self.op_path} && {self.report_command}")
#         threading.Thread(target=os.system, args=(f"cd {self.op_path} && {self.report_command}",)).start()
#         while (not os.path.exists(f"{self.op_path}/report.xml")):
#             pass
#         print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.op} was completed")
#         self.op_completed_path = f"{self.op_path}_{int(time.time() - process_start_time)}s_completed"
#         os.rename(self.op_path, self.op_completed_path)
#         return True



def generate_ci_data():
    data = {}
    with open(ci_report_file, 'r') as file:
        soup = BeautifulSoup(str(file.read()), 'html.parser')
    tbody = soup.findAll('tbody')[1]
    for op_tr in tbody.findAll('tr'):
        op = op_tr.findAll('th')[0].text
        template_td = op_tr.findAll('td')[report_plugin_position]
        data[op] = {result: 'untested' for result in ci_results}
        for result in data[op].keys():
            if len(template_td.find_all('span')) == 4:
                data[op][result] = int(re.findall(fr"{result[0].upper()}:(\d+)", template_td.text)[0])
    
    return data

# def get_from_TensorIterator_report(op: str):
#     result = 'TensorIterator not tested'
#     folders = [folder for folder in os.listdir(results_path) 
#         if folder.split('_')[0] == 'TensorIterator' and folder.split('_')[-1] == 'completed']
#     if folders:
#         report_path = f"{results_path}/{sorted(folders)[-1]}/report.xml"
#         with open(report_path, 'r', encoding='utf-8') as file:
#             re_str = rf'<{op}-\d+ passed="\d+" failed="\d+" skipped="\d+" crashed="\d+" passrate="\d+\.\d+" />'
#             result = ''.join(re.findall(re_str, file.read()))    
#     return result

def not_all_ops():
    # TODO: Delete operations that have 100% passrate in both lists, a better algorithm
    bad_ops = []
    for op in ops:
        flag = False
        if op in run_data:
            for result in run_results:
                if result not in ['passed']:
                    if run_data[op][result] != 'untested' and int(run_data[op][result]) > 0:
                        flag = True
                        break
        if op in ci_data:
            for result in ci_results:
                if result not in ['passed']:
                    if ci_data[op][result] != 'untested' and int(ci_data[op][result]) > 0:
                        flag = True
                        break
        if flag:
            bad_ops.append(op)
    ops = bad_ops
    
# [Black] [Blue] [Cyan] [Green] [Magenta] [Red] [White] [Yellow]
def get_op_results(run_data, ci_data, op):
    color, text, group = None, '', ''
    if op in run_data and op in ci_data:
        valid_data = True
        for run_result in run_results:
            if not isinstance(run_data[op][run_result], int):
                valid_data = False
                break
        for ci_result in ci_results:
            if not isinstance(ci_data[op][ci_result], int):
                valid_data = False
                break
        if valid_data:
            run_sum = 0
            for run_result in run_results:
                run_sum += run_data[op][run_result]
            run_passed = run_data[op]['passed']
            ci_sum = 0
            for ci_result in ci_results:
                ci_sum += ci_data[op][ci_result]
            ci_passed = ci_data[op]['passed']
            if run_sum == ci_sum and run_passed == ci_passed and run_sum == run_passed:
                color, text, group = '#D2E4D6', 'All tests passed', 'None'
    else:
        if op in ci_data:
            ci_untested = True
            for ci_result in ci_results:
                if ci_data[op][ci_result] != 'untested':
                    ci_untested = False
                    break
            if op not in run_data and ci_untested:
                color, text, group = '#D6C1F9', 'No tests', 'None'
            
    return color, text, group
        
    

def generate_xlsx():
    columns = ['Operation', 'Opset', 'ManualInfo', 'AutoInfo', 'Group'] + \
                [f"RUN {result}" for result in run_results] + \
                [f"CI {result}" for result in ci_results] + ['RUN logs']
    columns_len = [len(column) for column in columns]
    run_data = generate_run_data()
    ci_data = generate_ci_data()
    ops = sorted(list(set(list(run_data.keys()) + list(ci_data.keys()))))
    for op in ops:
        if op in ci_data and op not in run_data and op.split('_')[0] in time_wasting_ops:
            print(op)
            run_data[op] = {result: 'TIME OUT' for result in run_results}
            

    # not_all_ops()

    workbook_path = f"{results_path}/table_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.xlsx"
    workbook = xlsxwriter.Workbook(workbook_path)
    worksheet = workbook.add_worksheet('report')
    for i, column in enumerate(columns):
        worksheet.write(0, i, column, workbook.add_format({"bold": True}))
    for i, op in enumerate(ops):
        op_color, op_info, op_group = get_op_results(run_data, ci_data, op)
        # print(op, op_color, op_info, op_group)
        cell_format = workbook.add_format({'bg_color': op_color}) if op_color else None
        shift = 0
        worksheet.write(i + 1, shift, op)
        columns_len[shift] = max(columns_len[shift], len(str(op)))
        shift += 1
        worksheet.write(i + 1, shift, int(op.split('-')[-1]))
#         shift += 1
#         data = get_from_TensorIterator_report(op)
#         if data is not None:
#             worksheet.write(i + 1, shift, data)
        shift += 2
        worksheet.write(i + 1, shift, op_info)
        shift += 1
        worksheet.write(i + 1, shift, op_group)
        shift += 1
        if op in run_data:
            for j, result in enumerate(run_results):
                worksheet.write(i + 1, j + shift, run_data[op][result], cell_format)
        else:
            for j, result in enumerate(run_results):
                worksheet.write(i + 1, j + shift, 'untested', cell_format)
        shift += len(run_results)
        if op in ci_data:
            for j, result in enumerate(ci_results):
                worksheet.write(i + 1, j + shift, ci_data[op][result], cell_format)
        shift += len(ci_results)
        if op in logs_files:
            with open(logs_files[op], 'r', encoding='utf-8') as file:
                worksheet.write(i + 1, shift, ''.join(file.readlines()), cell_format)


    for i, column_len in enumerate(columns_len):
        if column_len > 0:
            worksheet.set_column(i, i, column_len)
    workbook.close()

def generate_full_report():
    if not os.path.exists(results_path):
        return
    folders = [folder for folder in os.listdir(results_path) if folder.split('_')[-1] == 'completed']
    folders = sorted(folders)
    ops = [folder.split('_')[0] for folder in folders]
    unique_folders, unique_ops = [], []
    for i, folder in enumerate(folders):
        if ops[i] not in ops[i+1:]:
            unique_folders.append(folder)
            unique_ops.append(ops[i])
    reports_path = f"{results_path}/reports"
    if os.path.exists(reports_path):
        shutil.rmtree(reports_path)
    os.makedirs(reports_path)
    for i, folder in enumerate(unique_folders):
        shutil.copy(f"{results_path}/{folder}/report.xml", f"{reports_path}/report_{unique_ops[i]}.xml")
    full_report_name = f"full_report_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}_{len(unique_ops)}pcs.xml"
    gen_report_command = f"{python_exec} {work_path}/merge_xmls.py -i {reports_path} -o {results_path}"
    os.system(gen_report_command)
    os.rename(f"{results_path}/report.xml", f"{results_path}/{full_report_name}")
    shutil.rmtree(reports_path)
    return f"{results_path}/{full_report_name}"
    
# TODO: get report_file by last name in work path
def generate_run_data(logs=False, ops=all_ops):
    data = {}
    for i, op in enumerate(sorted(ops)):
        run_folders = [folder for folder in os.listdir(results_path) 
                    if re.findall(fr"^{op}_.+_completed$", folder)]
#         data[op] = {result: 'untested' for result in run_results}
        if run_folders:
            run_path = f"{results_path}/{sorted(run_folders)[-1]}"
            for ir_run_folder in os.listdir(run_path):
                op_opset = re.findall(fr"{op}_opset\d+", ir_run_folder)[0]
                if op_opset not in data:
                    data[op_opset] = {result: 0 for result in run_results}
                report_path = f"{run_path}/{ir_run_folder}/report.xml"
                # print(report_path)
                if os.path.exists(report_path):
                    with open(report_path, 'r') as report_file:
                        xml_data = str(report_file.read())
                    passed_pattern = fr'<{op}-\d+ implemented=".+" passed="1" failed="0" skipped="0" crashed="0"'
                    failed_pattern = fr'<{op}-\d+ implemented=".+" passed="0" failed="1" skipped="0" crashed="0"'
                    skipped_pattern = fr'<{op}-\d+ implemented=".+" passed="0" failed="0" skipped="1" crashed="0"'
                    crashed_pattern = fr'<{op}-\d+ implemented=".+" passed="0" failed="0" skipped="0" crashed="1"'
                    if re.findall(passed_pattern, xml_data):
                        data[op_opset]['passed'] += 1
                    elif re.findall(failed_pattern, xml_data):
                        data[op_opset]['failed'] += 1
                        print(ir_run_folder, 'failed') if logs else None
                    elif re.findall(skipped_pattern, xml_data):
                        data[op_opset]['skipped'] += 1
                        print(ir_run_folder, 'skipped') if logs else None
                    elif re.findall(crashed_pattern, xml_data):
                        data[op_opset]['crashed'] += 1
                        print(ir_run_folder, 'crashed') if logs else None
                    else:
                        data[op_opset]['empty'] += 1
                        print(ir_run_folder, 'empty') if logs else None
                else:
                    data[op_opset]['empty'] += 1
                    print(ir_run_folder, 'empty') if logs else None
    result_data = {}
    for op, results in data.items():
        result_data[op.replace('_opset', '-')] = results
                    #print(f"REPORT??? {report_path}") # change
#             data[op] = {}
#             for result in run_results:
#                 result_path = f"{run_path}/gtest-parallel-logs/{result}"
#                 data[op][result] = len(os.listdir(result_path)) if path.exists(result_path) else 0

#             failed_report = f"{run_path}/{op}_failed_logs_result.txt"
#             if path.exists(failed_report):
#                 logs_files[op] = failed_report
    return result_data

if __name__ == '__main__':
    ops = [op for op in all_ops if op not in skipped_ops + completed_ops + time_wasting_ops]
#     ops = time_wasting_ops
#     ops = ['Add', 'Abs']
#     ops = []
    
    print(f"Ops for run: {ops}")
    
    for op in ops:
        GTestParallel(op).run(4 * 60)
#         GTestParallel(op).run(90 * 60)

    
#     run_data = generate_run_data(True)
#     run_data = generate_run_data(True, ['DetectionOutput'])
#     run_data = generate_run_data()
#     ci_data = generate_ci_data()
    
#     print(run_data)
#     print(ci_data)

    generate_xlsx()
    
#     for op, d in data.items():
#         if type(d['passed']) == int and d['passed'] > 0:
#             print(op)
#     print(data)
    
#     generate_full_report()
#     generate_xlsx()
    
#         generate_xlsx()
#     generate_xlsx()
        
#     for op in run_data:
#         if op not in ci_data:
#             print(op)


In [ ]:
for op in all_ops:
    run_folders = [folder for folder in os.listdir(results_path) 
                   if re.findall(fr"{op}_.+_completed", folder)]
    if run_folders:
        run_folder = sorted(run_folders)[-1]
        good_tests, all_tests = re.findall(fr"{op}_.+_(\d+)_of_(\d+)_\d+s_completed", run_folder)[0]
        if good_tests != all_tests:
            print("TIME OUT: ", run_folder)
    
    

In [ ]:
print(get_precision_irs_dict('Add'))

In [ ]:
os.system("$(pgrep -f conformanceTests)")

In [ ]:
number = 4
data = {}
with open(report_file, 'r') as file:
    soup = BeautifulSoup(str(file.read()), 'html.parser')
tbody = soup.findAll('tbody')[1]
for op_tr in tbody.findAll('tr'):
    op = op_tr.findAll('th')[0].text.split('-')[0]
    template_td = op_tr.findAll('td')[number]
    print(op)
    data[op] = {result: 'untested' for result in ci_results}
    for result in data[op].keys():
        print(result)
        if len(template_td.find_all('span')) == 4:
            data[op][result] = int(re.findall(fr"{result[0].upper()}:(\d+)", template_td.text)[0])
print(data)
# return data